# Get Incidents and Facilities for each interest area

Cada área de interesse inclui incidentes e facilities especificos

- Relação ia-facilitie está numa shapefile;
- Relação ia-incidents está numa tabela xlsx e pretende-se seleccionar os dados da shapefile;
- A escala da relação é a freguesia

In [ ]:
facilities     = '/mnt/disk1/jasp/rms/hospitais2022.shp'
idia = 'idai'

incidents      = '/mnt/disk1/jasp/rms/bgrirms_pnt.shp'
fregpk = 'idfreg'

relia_in       = '/mnt/disk1/jasp/rms/areainf_hsp_2021.xlsx'
relia_in_sheet = 'freg_hsp'
relidia = 'id_hsparea'
relfregfk = 'idfregr'

outfolder = '/mnt/disk1/jasp/rms/cf_inputs'

In [ ]:
import pandas as pd
import os
from glass.rd.shp import shp_to_obj
from glass.rd import tbl_to_obj
from glass.pys.oss import mkdir
from glass.wt.shp import df_to_shp

In [ ]:
# Obter as facilities
fdf = shp_to_obj(facilities)

# Get incidents
idf = shp_to_obj(incidents)

In [ ]:
idf

In [ ]:
# Obter grupo de facilities

ias = fdf[idia].unique()
ias.sort()

In [ ]:
ias

In [ ]:
# Obtêm dados relação freguesias - áreas de interesse

rdf = tbl_to_obj(relia_in, sheet=relia_in_sheet)

In [ ]:
rdf

In [ ]:
for ia in ias:
    # Identificar freguesias associadas a cada grupo de facilities
    idfreg = rdf[rdf[relidia] == ia]
    
    fregs = idfreg[relfregfk].unique()
    fregs.sort()
    
    if not len(fregs):
        continue
    
    # Select incidents
    s_idf = idf[idf[fregpk].isin(fregs)]
    
    if not s_idf.shape[0]:
        continue
    
    # Create folder to store results
    iafolder = mkdir(os.path.join(outfolder, f'ia_{str(ia)}'))
    
    # Export incidents
    df_to_shp(s_idf, os.path.join(iafolder, 'incidents.shp'))
    
    # Select facilities
    s_fdf = fdf[fdf[idia] == ia]
    
    # Export facilities
    df_to_shp(s_fdf, os.path.join(iafolder, 'facilities.shp'))